<center>

## <font color='blue'>ASTR 21100/31200</font>

## <font color='blue'>Generating (pseudo)-random numbers
   
 </center>
<p><p>
    


In many scientific and statistics applications (in particular in a class of methods called [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method)), sequences random numbers are needed. Here we discuss the general principles of how such numbers can be produced by a computer algorithm and give example of a simple yet powerful and very random class of random number generator algorithms called Permuted Congruential Generators (PCGs). 

### <font color='blue'> "Truly" uniform random number sequences exist</font>

<a href="http://www.fourmilab.ch/hotbits/">HotBits</a> generates random numbers using inherent uncertainty of quantum systems by using Geiger counter counts of radiactive source, while <a href="https://www.random.org/">random.org</a> uses atmospheric radio noise to produce random numbers. 

You can use these services to get truly random  sequences. However, this is essentially almost never needed in practice, as periods and statistical properties of good modern pseudo random number generators are extremely large and are sufficient for all practical purposes.

Plus, using these sequences also has its own set of issues related to some existing correlations of generated random numbers. Also, a practical issue is that it's very difficult to generate very large sequences of random numbers this way. 

### <font color='blue'> Pseudo-Random Number Generators (PRNGs)</font>

A PRNG is a function that takes as input one or more numbers and uses an algorithm to produce the next *pseudo-random* number based on these inputs. 

### <font color='blue'> The argument for treating numbers generated by a deterministic PRNGs as random</font>

The way to think about it is by analogy with *artificial intelligence*. The famous <a href="https://www.turing.org.uk/scrapbook/test.html">Turing test</a> posits that we've created artificial intelligence if computer can chat with a person who would not be able to tell whether they are chatting with a computer or a human just based on the words in the conversation.  

Likewise, "randomness" of a PRNG should be judged based on whether someone can tell that the sequence of numbers is not random without knowing how the sequence was obtained. 
    
If a person cannot determine by any test that data is not random, then it is effectively random, even if it was produced by a deterministic algorithm/code. ***Note, however,*** that this assumes that stringent tests will be used here, not some superficial examination. 

For scientific applications this is the only criterion that matters. For applications in cryptography there are additional criteria for a good PRNG. The main one that it should be very difficult (or close to impossible) to reverse engineer the algorithm used to produce a sequence of pseudo-random numbers. 

For some scientific and cryptography applications speed and ability to produce very long sequences of pseudo-random numbers with good "randomness" properties is also a big factor.

### <font color='blue'> Linear congruential PRNGs</font>

Pseudo-random numbers from $0$ to $m$ are generated via *recursion* formula:

$$x_{n+1}=(ax_n+c)\, \mathrm{mod}\, m$$

where $x_0$ is the starting integer number, called *the seed*. 

Here $\mathrm{mod}\, m$ is the <a href="https://en.wikipedia.org/wiki/Modulo_operation">modulo operation</a> (remainder of the integer division by $m$, operator % in Python) and $m$ is intended period of the sequence (numbers larger than $m$ are "wrapped around" into $[0,m]$ range). 

**_Note_**: Historically, *linear-congruential* PRNG algorithms were popular in early applications in the 1950s-1970s due to their simplicity, speed, and low memory requirements. They are still the fastest PRNG algorithms. 

**_Note_**: There is a formal mathematical justification for such PRNGs based on the Kronecker-Weyl theorem according to which all possible numbers from 0 to $m$ can be generated via the recursion formula above. 

A simple implementation of the Linear Congruential Generator (LCG) is below.

In [1]:
def lcg(xr, a=2733, m=2**31-1):
    return (a*xr) % m # That's it folks!

Note that we defined two so-called *keyword* arguments in the function above. The first argument, <tt>xr</tt>, is a usual function argument, but the other two (<tt>a, m</tt>) are defined with assigned default values. When parameters are defined like this, we can call the functions, without specifying them explicitly:

In [2]:
xr = 2449
lcg(xr)

6693117

At the same time we can changed them at will, when we call the function

In [3]:
a=29383
m=2**32
lcg(xr, a, m)

71958967

or as follows:

In [4]:

lcg(xr, a=29383, m=2**32)

71958967

when we do this like this, we don't even need to respect the order of these keyword arguments defined in the function, because they are identified by the function by their names (this is only true for keyword arguemnts, not for regular function arguments). For example, this works just as well 

In [5]:
lcg(xr, m=2**32, a=29383)

71958967

but this does not work (all keyword arguments must follow all of the non-keyword (positional) arguments, because the latter are identified only by their position, not by their name):

In [7]:
lcg(m=2**32, xr, a=29383)

71958967

 The loop below shows the numbers produced at each iteration of LCG and their binary form (on the right) using built-in function <tt>bin()</tt> which turns an integer into a string of 0s and 1s that contains binary representation of this integer. Look carefully at the few leftmost and a few rightmost bits. Do they look random to you?  

In [8]:
xr = 2736273
for i in range(40):
    xr = lcg(xr, a=65539, m=2**31)
    print('{:>10d}  {:>32s}'.format(xr, bin(xr)[2:]))

1091453363   1000001000011100100000110110011
  81642777       100110111011100010100011001
1404194635   1010011101100100100111101001011
1247931873   1001010011000011110110111100001
1292290467   1001101000001101100100110100011
 817323241    110000101101110101110011101001
1863259835   1101111000011110001011010111011
1676166193   1100011111010000100010000110001
1877593235   1101111111010011100110010010011
 475030969     11100010100000110010110111001
 984232235    111010101010100011000100101011
1630114689   1100001001010011001001110000001
 922598019    110110111111011011101010000011
1601974153   1011111011111000010111110001001
1308462747   1001101111111011000111010011011
2022943697   1111000100100111010101111010001
 361497459     10101100011000000001101110011
1142360665   1000100000101110000101001011001
1453203211   1010110100111100001111100001011
 585456929    100010111001010101110100100001
1171330915   1000101110100010001011101100011
1758873129   1101000110101100100011000101001
  11260539

However, how bad the regularity is depends on the choice of $a$ and $m$. For example, repeat the run with the default values: <tt>a=2733, m=2**31-1</tt> and see if you can still find regularities visually. 

In [9]:
def lcg(xr, a=2733, m=2**31-1):
    return (a*xr) % m

### <font color='blue'>Good choices of constants for LCGs</font>

Even better results will be achieved if before permutation, we use optimal values of $a$, $c$ and $m$ that provide the best LCG sequences. Combinations of these are listed in the paper by <a href="http://www.ams.org/journals/mcom/1999-68-225/S0025-5718-99-00996-5/S0025-5718-99-00996-5.pdf">L'Ecuyer (1999)</a> (see Table 2 in that paper for values of $m$ and $a$, $c=0$ in all of the cases). 

For example, $m=2^{31}-1$ and $a=1583458089$ suitable for 32-bit LCG, while 
$m=2^{63}-25$ and $a=4645906587823291368$ can be used for a 64-bit LCG.

### <font color='blue'>Making functions "remember" the previous value of random number $x_{n-1}$</font>

This material in this section is optional and can be skipped if you are new to programming and Python. Nevertheless,  it is useful to know that such possibility exists for future reference. So skimming it is recommended. 

One way to do this is to declare variable <tt>xr</tt> used in the function as a *global* variable. 

    global xr 
    
However, this makes such variables accessible from anywhere in the code. When a code gets complex, it makes such variables prone for inadvertent overwrites and errors (bugs) during execution. So global variables should be used sparingly and for variables that are not expected to be changed and good names, that are unlikely to be defined by someone else elsewhere in the code. 

There is a different, better way. 

To make variable accessible across different function calls, we can use function attribute assignment. Function attribute is a variable that starts with a function name, dot, and variable name. For example, to set function attribute variable x for function <tt>func</tt> we do (in order for this to work function <tt>func</tt> must already be defined somewhere):

    func.x = 10

Using function attributes allows access to values computed within function outside of the function, as though they are global variables, but it creates a unique name-space for these variables, by associating them with function name (in the example below, outside the function <tt>lcg2.xr</tt> can also be accessed using that name, never as <tt>xr</tt>). This prevents unintentional errors of changing a global variable name inadvertently in the external code due to a name conflict. 

In [10]:
from random import randint

def lcg2(a=2733, m=2**31-1, seed=None):
    '''
    LCG version which self-initializes and does not require x to be passed. 
    Parameters: a - int, multiplicative factor
                m - int, sequence period
                seed = positive int, seed number (can be ommitted for random self-initialization)
                
    Returns: 
            xr - int, random number generated by LCG with current a and m
    '''
    try: 
        lcg2.xr = (a*lcg2.xr) % m
    except:
        if seed is None: 
            # random seed
            lcg2.xr = randint(0,100000) + 9323 # addition to avoid 0
        else:
            lcg2.xr = seed # seed with a specified number
    return lcg2.xr
        

In [11]:
for i in range(10):
    # for lcg2 we don't have to supply an argument
    # lcg2.xr is also available outside the function
    print(lcg2(), lcg2.xr ) 


72795 72795
198948735 198948735
413530064 413530064
601266590 601266590
436600515 436600515
1375783410 1375783410
1919677280 1919677280
175456619 175456619
634086446 634086446
2086437436 2086437436


**_Note_**: such attribute assignment is somewhat more expensive that regular assignment and so if the function is run hundreds of millions of time, such choice of coding may not result in the fastest execution. You can read about various uses (and abuses) of the function attribute variables [here](https://stackoverflow.com/questions/338101/python-function-attributes-uses-and-abuses), if you are curious. 

### <font color='blue'>Do deficiencies of LCGs matter in computations?</font>

It depends on applications, how many random numbers are generated, and what accuracy you need to achieve... In general, it is always worth to use random number generator functions that was proven to have best randomness, so as not worry about such issues at all. 

## <font color='blue'>Permuted Congruential Generators (PCGs)</font>

Professor <a href="https://www.cs.hmc.edu/~oneill/index.html">Melissa O'Neill</a> of the Harvey Mudd college proposed a powerful, yet simple way to fix the well-understood flaws of LCGs in 2014. 
The idea is to permute (or scramble) bits of the integer numbers that are generated by LCG using a permutation function. 
<p>    
<center>
<img width=300 src="https://www.cs.hmc.edu/~oneill/images/melissa1.jpg"></img></center>    


In [12]:
print("illustration of a xor operation:")
i1, i2 = 98293489442, 99278249448
print(bin(i1)[2:])
print(bin(i2)[2:])
print("i1 xor i2:")
print ("    %s"%bin(i1 ^ i2)[2:])
print('-----------------------------------')
print("illustration of a right xorshift operation:")
print("i1 shifted by 3 bits to the right:")
print("   %s"%bin(i1 >> 3)[2:])
print("i2 shifted by 3 bits to the right:")
print("   %s"%bin(i2 >> 3)[2:])
print("i1 shifted by 3 bits to the right and xor'ed with i2:")
print ("%s"%bin((i1 >> 3) ^ i2)[2:])

illustration of a xor operation:
1011011100010101111111001111100100010
1011100011101011100011101110111101000
i1 xor i2:
    111111111110011100100001011001010
-----------------------------------
illustration of a right xorshift operation:
i1 shifted by 3 bits to the right:
   1011011100010101111111001111100100
i2 shifted by 3 bits to the right:
   1011100011101011100011101110111101
i1 shifted by 3 bits to the right and xor'ed with i2:
1010111000001001001100010111000001100


This permutation function randomizes the lower 10-15 bits of the LCG numbers. 

The specific example function below are based on Melissa O'Neill's implementation (numbers below refer to the 32-bit version for simplicity) - no need to understand details here, you should just understand the main idea of what such permutations do:

1. "random xorshift" operation. This operation uses the three highest (leftmost) bits of the number to choose by how many bits to shift to the right. The three upper bits if they are random will produce random integer numbers from 0 to 7. This is a bit too small - the number of lower non-random bits in LCG can be larger than this. So we add 4 to the number given by the three upper bits to make resulting random number to vary from 4 to 11.


2. The original LCG number is then bit-shifted to the right by this random number: i.e. we are dropping a random number (from 4 to 11) of lower bits of the LCG number, which usually show the worst irregularities. The remaining bits should be pretty random. 


3. The random number from step 2 is xor'ed with the original LCG number. Note that the right shift in step 2 created a random number of zeros in the upper bits of the resulting number. So xor'ing original LCG number with those zeros will retain the upper bits of the original number, which are almost always very random (for good choices of $a$ and $m$). 
Lower bits of the shifted number should be pretty random after the shift, because they are a random subset of the original upper bits of the LCG number. Xor'ing these random bits with lower bits of the original LCG number will randomize them. 


4. If this was not enough, the algorithm multiplies the result of step 3 by a large odd integer number to create a random number with more non-zero bits. Then it drops lowest bits of the result by shifting it by 22 bits to the right and then xor's the result with the result of step 3 again in the second cycle of randomizing the lowest bits. 

*The code in the cell below is not meant to be parsed/understood by you.* It's not too complicated, but it uses NumPy, which was not covered yet. If you do look at it, just know that NumPy can deal with lists as though they are variables operating on all elements at once (this is called *vector operation*), without arranging loops to scan through list elements. 

The function <tt>np.uint32</tt> simply converts integer numbers to integers limited to 32-bits for internal purposes. 

In [13]:
import numpy as np
u4 = np.uint32(4)
u22 = np.uint32(22)
u28 = np.uint32(28)
u277803737 = np.uint32(277803737)

def permute_bits_rxs_m_xs_32_32(rnd) :  
    """
    a function performing permutation of bits in a 32-bit integer
    based on rxs_m_xs_32_32 implementation 
    by Prof. Melissa O'Neill
    http://www.pcg-random.org/download.html

    Parameters:
    -----------
    a 1d vector of 32-bit integers
    
    Returns:
    --------
    a 1d vector of 32-bit integers with bits scrambled by 
    the rxs_m_xs_32_32 permutation function, 
    see Sections 6.3.4 and 5.5.1 in M. O'Neill's paper
    """
    # first 2 steps: random xorshift and multiplication
    # shift bits in 32-bit number by 28 to the left, which leaves 3 upper bits
    # then add 4 to the results and shift the bits in a number by that result 
    # to the right, finally xor result with the number itself
    # then multiply by a good large number to randomize top bits
    rnd = np.uint32(rnd)
    rndd = np.uint32((rnd >> ((rnd >> u28) + u4)) ^ rnd) * u277803737;
    # third step: shift bits resulting from first 2 steps 
    # by 22 positions to the right and xor result with the result of step 2
    return (rndd >> u22) ^ rndd;

The code below records <tt>nrnd</tt> integers generated with the LCG with very bad choice of $a$ and $m$ (the [infamous RANDU generator](https://en.wikipedia.org/wiki/RANDU)), which was obviously non-random from just examination of bits in numbers it produces. 

It then passes them through the function above that implements one of many possible PCG choices to scramble lowest bits of each integer. 

In [14]:
seed = 392382

# permute bits of RANDU generated numbers
nrnd = 10000
period = 2**31

# the infamously BAD RANDU RNG:
a = 65539

xr = seed
rv = []
for i in range(nrnd): 
    xr = lcg(xr, a=a, m=period)
    rv.append(xr) # append xr to the list rv
    
# about to be much improved by additional scrambling of bits in numbers it generated
rvp32 = permute_bits_rxs_m_xs_32_32(rv) 

# turn resulting integers into integers within [0,period]
rvi = []
rvf = [] # initialize list rvf for random floats

for rd in rvp32: 
    dummy = rd % period # wrap integers into [0,period] range
    rvi.append(dummy) 
    # turn integers into floats in range [0,1]
    rvf.append(dummy / period) # dummy/period are floats in [0,1]

The numbers with the scrambled bits no longer have obvious patterns in their bits. 

In [15]:
nrnd = 10 
for dummy in rvi[:nrnd]:
    print("binary form: %32s"%(bin(dummy)[2:]))

binary form:  1100111011111100000011010111001
binary form:  1101100000110101000101011010110
binary form:  1001010001111110101100100110010
binary form:  1010010010010110000010100000111
binary form:  1110100000000011101111100101110
binary form:  1011000101100100100001010001100
binary form:   100110011010010001111101000000
binary form:  1001100011101011001001011001111
binary form:  1010100000000100000000010101000
binary form:  1101001010010001010111010011000


### <font color='blue'>Ok, so we have random integers from $0$ to $m$, how do we get real numbers?</font>
    
By a simple rescaling. To get $x\in [0,1]$ from the integers generated above, we simply do $x = i/{\rm period}$, as is done to produce list of random floats <tt>rvf</tt> above. 
    
If we want to get real (floating point) numbers in an interval $[a,b]$, we get them by rescaling integers $i$ generated by a PRNG with a given period:
    
$$x = (b-a)\times i/{\rm period} + a,\ \ \ \ \ x\in [a,b].$$

In [16]:
for i in range(10):
    print(rvf[i])

0.8085335162468255
0.8445600075647235
0.5800582403317094
0.6429144176654518
0.906307122670114
0.6929400619119406
0.30008307099342346
0.5973380575887859
0.6563111133873463
0.8225305788218975


In [17]:
#Rescale to the interval $[1,2]$
for i in range(10):
    print((2-1)*rvf[i] + 1)


1.8085335162468255
1.8445600075647235
1.5800582403317094
1.6429144176654518
1.906307122670114
1.6929400619119406
1.3000830709934235
1.5973380575887859
1.6563111133873463
1.8225305788218975


### <font color='blue'>How random are the numbers generated by PCGs? </font>

Very random. In fact, PCG generators [pass all known tests of randomness](https://www.johndcook.com/blog/2017/07/07/testing-the-pcg-random-number-generator/). 
    
    
It's more than sufficiently random to use for any scientific analyses that require random numbers and [Monte Carlo experiments](https://en.wikipedia.org/wiki/Monte_Carlo_method). Most PCG variants are not good for cryptography (although some variants can be used), but cryptography is of no concern in science or statistics. 

Python's [<tt>random</tt> module](https://www.w3schools.com/python/module_random.asp) uses an older  PRNG algorithm based on the Mersenne Twister method. This method is much more complicated but PRNG based on it is not really better. However, the random submodule of the NumPy module for fast numerical computations, which we will start studying and using in the near future, uses Melissa O'Neill's PCG generators as a default PRNG. 

### <font color='blue'>Take-home message</font>

It's not completely obvious (it took people ~60 years of research!), but PRNGs that are fast, simple (few lines of code), idea  and have great statistical properties (i.e. very close to being truly random) can be implemented  based on Melissa O'Neill's Permuted Congruential Generators.


**_Note_**: floating point numbers produced with PRNGs are said to be *distributed uniformly* in the interval for which they are generated because probability to fall within any equal size subinterval with the range is the same. We say that the random numbers are *drawn from a uniform probability distribution function* (pdf). We will discuss probability distribution functations (aka probability density functions) next. 